In [27]:
!pip install fastembed
!pip install qdrant-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 8.8 MB/s eta 0:00:0031m9.0 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from fastembed import TextEmbedding

/Users/phonghoang/Desktop/LLM Zoomcamp/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [11]:
model = TextEmbedding(model_handle)
query = "I just discovered the course. Can I join now?"
embedding_1 = list(model.embed([query]))[0]

In [12]:
value_1 = min(embedding)
print(value_1)

-0.12396319741919225


In [13]:
doc = 'Can I still join the course after the start date?'

In [14]:
embedding_2 = list(model.embed([doc]))[0]

In [15]:
embedding_1 @ embedding_2

np.float64(0.9008528856818037)

In [16]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [20]:
from numpy.linalg import norm

cos_sim = []
for doc in documents:
    doc_vec = list(model.embed([doc['text']]))[0]
    similarity = embedding_1 @ doc_vec / (norm(embedding_1) * norm(doc_vec))
    cos_sim.append(similarity)
cos_sim

[np.float64(0.7629684493123693),
 np.float64(0.8182378325403558),
 np.float64(0.8085397290762828),
 np.float64(0.713307860072464),
 np.float64(0.7304499388234832)]

In [21]:
cos_sim = []
for doc in documents:
    doc_vec = list(model.embed([doc['question'] + ' ' + doc['text']]))[0]
    similarity = embedding_1 @ doc_vec / (norm(embedding_1) * norm(doc_vec))
    cos_sim.append(similarity)
cos_sim

[np.float64(0.8514543121225202),
 np.float64(0.8436594058703555),
 np.float64(0.8408287206920104),
 np.float64(0.7755157711899687),
 np.float64(0.8086008033213788)]

In [30]:
list_models = TextEmbedding.list_supported_models()
list_dim = [model['dim'] for model in list_models]
min(list_dim)

384

In [22]:
model = TextEmbedding('BAAI/bge-small-en')
query = "I just discovered the course. Can I join now?"
embedding_1 = list(model.embed([query]))[0]
embedding_1.shape

Fetching 5 files: 100%|███████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.08s/it]


(384,)

In [23]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [24]:
texts = [doc['question'] + " " + doc['text'] for doc in documents]

In [25]:
embeddings = list(model.embed(texts))

In [31]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333/")

collection_name = "zoomcamp-rag"

# Create collection with the correct vector size (384)
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
)

/var/folders/_g/lbw68g6x4db_w26lhwhvvkgr0000gn/T/ipykernel_80251/1650619499.py:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [33]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding

model_handle = TextEmbedding(model_name="BAAI/bge-small-en")
client = QdrantClient()

points = []
id = 0

for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        full_text = doc['question'] + ' ' + doc['text']
        vector = list(model_handle.embed([full_text]))[0]  # get vector

        point = models.PointStruct(
            id=id,
            vector=vector,
            payload={
                "text": doc['text'],
                "question": doc['question'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(point)
        id += 1

# Upsert into Qdrant
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [34]:
query_text = "I just discovered the course. Can I join now?"
query_embedding = list(model.embed([query_text]))[0].tolist()

# Search top 1
search_results = client.search(
    collection_name=collection_name,
    query_vector=query_embedding,
    limit=1
)

top_score = search_results[0].score
print(f"Highest similarity score: {top_score}")

Highest similarity score: 0.8703172


/var/folders/_g/lbw68g6x4db_w26lhwhvvkgr0000gn/T/ipykernel_80251/1627881269.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
